<a href="https://colab.research.google.com/github/JUPUDI-VAMSI-KALYAN/Data_Science-Python-/blob/master/Kidney_disease_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded=files.upload()

Saving kidney.csv to kidney.csv


In [0]:
import pandas as pd
import numpy as np

In [0]:
df=pd.read_csv("kidney.csv")
df.head()

,48,80,1.020,1,0,?,normal,notpresent,notpresent.1,121,36,1.2,?.1,?.2,15.4,44,7800,5.2,yes,yes.1,no,good,no.1,no.2,ckd
0,7,50,1.020,4,0,?,normal,notpresent,notpresent,?,18,0.8,?,?,11.3,38,6000,?,no,no,no,good,no,no,ckd
1,62,80,1.010,2,3,normal,normal,notpresent,notpresent,423,53,1.8,?,?,9.6,31,7500,?,no,yes,no,poor,no,yes,ckd
2,48,70,1.005,4,0,normal,abnormal,present,notpresent,117,56,3.8,111,2.5,11.2,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
3,51,80,1.010,2,0,normal,normal,notpresent,notpresent,106,26,1.4,?,?,11.6,35,7300,4.6,no,no,no,good,no,no,ckd
4,60,90,1.015,3,0,?,?,notpresent,notpresent,74,25,1.1,142,3.2,12.2,39,7800,4.4,yes,yes,no,good,yes,no,ckd


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 25 columns):
48              399 non-null object
80              399 non-null object
1.020           399 non-null object
1               399 non-null object
0               399 non-null object
?               399 non-null object
normal          399 non-null object
notpresent      399 non-null object
notpresent.1    399 non-null object
121             399 non-null object
36              399 non-null object
1.2             399 non-null object
?.1             399 non-null object
?.2             399 non-null object
15.4            399 non-null object
44              399 non-null object
7800            399 non-null object
5.2             399 non-null object
yes             399 non-null object
yes.1           399 non-null object
no              399 non-null object
good            399 non-null object
no.1            399 non-null object
no.2            399 non-null object
ckd             399 non-null ob

In [0]:
X=df.drop("ckd",axis=1)
y=df["ckd"]

In [0]:
# Import necessary modules
from sklearn_pandas import DataFrameMapper
from sklearn_pandas import CategoricalImputer

# Check number of nulls in each feature column
nulls_per_column = X.isnull().sum()
print(nulls_per_column)

# Create a boolean mask for categorical columns
categorical_feature_mask = X.dtypes == object

# Get list of categorical column names
categorical_columns = X.columns[categorical_feature_mask].tolist()

# Get list of non-categorical column names
non_categorical_columns = X.columns[~categorical_feature_mask].tolist()

# Apply numeric imputer
numeric_imputation_mapper = DataFrameMapper(
                                            [([numeric_feature],Imputer(strategy="median")) for numeric_feature in non_categorical_columns],
                                            input_df=True,
                                            df_out=True
                                           )

# Apply categorical imputer
categorical_imputation_mapper = DataFrameMapper(
                                                [(category_feature, CategoricalImputer()) for category_feature in categorical_columns],
                                                input_df=True,
                                                df_out=True
                                               )

48              0
80              0
1.020           0
1               0
0               0
?               0
normal          0
notpresent      0
notpresent.1    0
121             0
36              0
1.2             0
?.1             0
?.2             0
15.4            0
44              0
7800            0
5.2             0
yes             0
yes.1           0
no              0
good            0
no.1            0
no.2            0
dtype: int64


In [0]:
from sklearn.pipeline import FeatureUnion

# Combine the numeric and categorical transformations
numeric_categorical_union = FeatureUnion([
                                          ("num_mapper", numeric_imputation_mapper),
                                          ("cat_mapper", categorical_imputation_mapper)
                                         ])
Dictifier = X.to_dict("record")

In [0]:
# Create full pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
                     ("featureunion", numeric_categorical_union),
                     ("dictifier", Dictifier(X.to_dict())),
                     ("vectorizer", DictVectorizer(sort=False)),
                     ("clf", xgb.XGBClassifier(max_depth=3))
                    ])

# Perform cross-validation
cross_val_scores = cross_val_score(pipeline, X, y, scoring="roc_auc", cv=3)

# Print avg. AUC
print("3-fold AUC: ", np.mean(cross_val_scores))

TypeError: ignored